In [1]:
import numpy as np
import dask.dataframe as dd
import pandas as pd

from py.consts import DATA_DIR, DAYS_OF_WEEK, MONTHS_OF_YEAR
from py.utils import one_hot_encode

In [2]:
df = pd.read_parquet(DATA_DIR / "flitsers.parquet")
# df.compute()

In [3]:
print(f"number of records: {len(df)}")
frac: float = 0.5
df = df.sample(frac=frac)
print(f"sampled to {len(df)} records")
    
df.head()

number of records: 93412
sampled to 46706 records


,id,datum,soort_weg,wegnummer,zijde,hm_paal,type_controle,tijd_van_melden,details,laatste_activiteit,...,weer_temp_min,weer_luchtdruk_hpa,weer_luchtvochtigheid_procent,weer_windsnelheid_m_per_sec,weer_windrichting_graden,weer_bewolking_procent,weer_regen_mm,weer_sneeuw_mm,weer_zonnestand,weer_locatie_naam
53100,54436,2020-01-10,snelweg,A16,rechts,25.7,Radar,2020-01-10 07:21:00,bij knpt ridderkerk-noord\n</td,2020-01-10 13:50:25.836719,...,7.22,1011.0,93.0,3.6,230.0,75.0,3.31,NaN,voor_zonsopgang,Ridderkerk
91510,102862,2022-12-12,nweg,N313,beide,0,Radar,2022-12-12 09:44:43,net na de grensovergang\n</td,2022-12-12 14:50:21.076660,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
50223,50622,2019-09-25,snelweg,A15,rechts,47.2,Radar,2019-09-25 10:35:11,na de botlektunnel\n</td,2019-09-25 16:20:04.022813,...,13.89,1003.0,93.0,5.1,190.0,75.0,NaN,NaN,na_zonsopgang,Spijkenisse
4439,4293,2016-04-13,nweg,N351,beide,20.5,Radar,2016-04-13 18:39:47,on,2016-04-13 23:51:56.760000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
34728,34929,2018-06-14,nweg,N289,beide,0,Laser,2018-06-14 10:01:00,on,2018-06-14 18:30:03.738831,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [4]:
df.shape

(46706, 27)

In [5]:
dataset = df[["datum", "wegnummer", "zijde", "tijd_van_melden", "laatste_activiteit"]]

dataset["datum"] = pd.to_datetime(dataset.datum)

dataset["y"] = 1

dataset.head()

/var/folders/2b/jq4whb6n4r5777w6yjyyxy800000gn/T/ipykernel_43843/452800526.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["datum"] = pd.to_datetime(dataset.datum)
/var/folders/2b/jq4whb6n4r5777w6yjyyxy800000gn/T/ipykernel_43843/452800526.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["y"] = 1


,datum,wegnummer,zijde,tijd_van_melden,laatste_activiteit,y
53100,2020-01-10,A16,rechts,2020-01-10 07:21:00,2020-01-10 13:50:25.836719,1
91510,2022-12-12,N313,beide,2022-12-12 09:44:43,2022-12-12 14:50:21.076660,1
50223,2019-09-25,A15,rechts,2019-09-25 10:35:11,2019-09-25 16:20:04.022813,1
4439,2016-04-13,N351,beide,2016-04-13 18:39:47,2016-04-13 23:51:56.760000,1
34728,2018-06-14,N289,beide,2018-06-14 10:01:00,2018-06-14 18:30:03.738831,1


In [6]:
# for every date:
    # for every road with no records on this day:
        # create y=0 record with datum=datum, start time = 00:00:00 and stop_time = 23:59:59 and random values

all_roads = dataset.wegnummer.unique()

new_records = []

for i, date_ in enumerate(dataset.datum.unique()):
    positive_records_on_this_date = dataset.loc[(dataset.datum == date_) & (dataset.y == 1)]
    
    roads_with_records = list(positive_records_on_this_date.wegnummer.unique())
    roads_with_no_records = [road for road in all_roads if road not in roads_with_records]
    
    new_date_records = []
    
    fake_start_time = date_
    fake_end_time = date_.to_period("D").to_timestamp(how="end")
    
    for road in roads_with_no_records:
        for side in ["links", "rechts", "beide"]:
            new_record = {
                "y": 0,
                "datum": date_,
                "wegnummer": road,
                "zijde": side,
                "tijd_van_melden": fake_start_time,
                "laatste_activiteit": fake_end_time,
            }
            new_date_records.append(new_record)
    
    new_records.extend(new_date_records)
    
    print(f"date {date_}: added {len(new_date_records)} new records")

dataset = pd.concat([dataset, pd.DataFrame(new_records)])
print(f"Added {len(new_records)} new records")

date 2020-01-10 00:00:00: added 1659 new records
date 2022-12-12 00:00:00: added 1647 new records
date 2019-09-25 00:00:00: added 1608 new records
date 2016-04-13 00:00:00: added 1629 new records
date 2018-06-14 00:00:00: added 1608 new records
date 2020-03-17 00:00:00: added 1635 new records
date 2018-06-19 00:00:00: added 1596 new records
date 2017-07-24 00:00:00: added 1614 new records
date 2022-02-25 00:00:00: added 1638 new records
date 2020-04-26 00:00:00: added 1656 new records
date 2020-08-03 00:00:00: added 1650 new records
date 2020-08-11 00:00:00: added 1647 new records
date 2020-10-08 00:00:00: added 1647 new records
date 2022-11-04 00:00:00: added 1671 new records
date 2019-02-11 00:00:00: added 1614 new records
date 2017-07-22 00:00:00: added 1665 new records
date 2018-08-14 00:00:00: added 1626 new records
date 2020-04-27 00:00:00: added 1641 new records
date 2016-09-19 00:00:00: added 1680 new records
date 2021-07-12 00:00:00: added 1629 new records
date 2021-12-22 00:0

In [7]:
# for every date:
    # for every (positive) record:
        # create y=0 duplicate with start time = 00:00:00 and stop time = tijd_van_melden
        # create y=0 duplicate with start time = laatste_activiteit en stop time = 23:59:59

new_dfs = []
total_new_records_count = 0
all_sides = ["links", "rechts", "beide"]
        
for i, date_ in enumerate(dataset.datum.unique()):
    new_dfs_this_date = []
    positive_records_on_this_date = dataset.loc[(dataset.datum == date_) & (dataset.y == 1)]
        
    fake_start_time = date_
    fake_end_time = date_.to_period("D").to_timestamp(how="end")
    
    negative_records_before_start_time = positive_records_on_this_date.assign(y=0, tijd_van_melden=fake_start_time, laatste_activiteit=positive_records_on_this_date.tijd_van_melden)
    negative_records_after_start_time = positive_records_on_this_date.assign(y=0, tijd_van_melden=positive_records_on_this_date.laatste_activiteit, laatste_activiteit=fake_end_time)
    new_dfs_this_date.extend([negative_records_before_start_time, negative_records_after_start_time])
    
    for side in all_sides:
        records_per_side = positive_records_on_this_date.loc[positive_records_on_this_date.zijde == side]
        
        remaining_sides = [s for s in all_sides if s != side]
        for s in remaining_sides:
            new_records_other_side = records_per_side.assign(y=0, zijde=s, tijd_van_melden=fake_start_time, laatste_activiteit=fake_end_time)
            new_dfs_this_date.append(new_records_other_side)
    
    print(f"added {sum([len(_df) for _df in new_dfs_this_date])} fake records for date {date_} ({i} of {len(dataset.datum.unique())})")
    new_dfs.extend(new_dfs_this_date)

old_dataset_length = len(dataset)
dataset = pd.concat([dataset, *new_dfs])
new_dataset_length = len(dataset)        
    
print(f"\n added {new_dataset_length - old_dataset_length} negative records")   
    

added 56 fake records for date 2020-01-10 00:00:00 (0 of 2523)
added 64 fake records for date 2022-12-12 00:00:00 (1 of 2523)
added 144 fake records for date 2019-09-25 00:00:00 (2 of 2523)
added 108 fake records for date 2016-04-13 00:00:00 (3 of 2523)
added 148 fake records for date 2018-06-14 00:00:00 (4 of 2523)
added 88 fake records for date 2020-03-17 00:00:00 (5 of 2523)
added 156 fake records for date 2018-06-19 00:00:00 (6 of 2523)
added 112 fake records for date 2017-07-24 00:00:00 (7 of 2523)
added 84 fake records for date 2022-02-25 00:00:00 (8 of 2523)
added 60 fake records for date 2020-04-26 00:00:00 (9 of 2523)
added 76 fake records for date 2020-08-03 00:00:00 (10 of 2523)
added 72 fake records for date 2020-08-11 00:00:00 (11 of 2523)
added 68 fake records for date 2020-10-08 00:00:00 (12 of 2523)
added 40 fake records for date 2022-11-04 00:00:00 (13 of 2523)
added 124 fake records for date 2019-02-11 00:00:00 (14 of 2523)
added 44 fake records for date 2017-07-22 00

In [8]:
date = "2016-08-20"
print(dataset.loc[(dataset.datum == date) & dataset.y == 1])
road = "A16"
dataset.loc[(dataset.datum == date) & (dataset.wegnummer == road)]

          datum wegnummer  zijde     tijd_van_melden  \
5423 2016-08-20      N370  beide 2016-08-20 15:20:22   
5982 2016-08-20      N370  beide 2016-08-20 15:20:22   
5410 2016-08-20        N7  links 2016-08-20 09:09:14   
5426 2016-08-20      N307  beide 2016-08-20 20:46:26   

             laatste_activiteit  y  
5423 2016-08-20 18:40:05.672053  1  
5982 2016-08-20 22:55:07.769805  1  
5410 2016-08-20 09:40:06.790927  1  
5426 2016-08-20 22:50:05.853323  1  


,datum,wegnummer,zijde,tijd_van_melden,laatste_activiteit,y
2815002,2016-08-20,A16,links,2016-08-20,2016-08-20 23:59:59.999999999,0
2815003,2016-08-20,A16,rechts,2016-08-20,2016-08-20 23:59:59.999999999,0
2815004,2016-08-20,A16,beide,2016-08-20,2016-08-20 23:59:59.999999999,0


In [8]:
# dataset["year"] = dataset.datum.dt.year
dataset["month"] = dataset.datum.dt.month_name()
dataset["day"] = dataset.datum.dt.day_name()
dataset = dataset.drop(columns=["datum"])

In [9]:
# dataset.compute()
print(dataset.shape)
dataset.head()

(4385206, 7)


,wegnummer,zijde,tijd_van_melden,laatste_activiteit,y,month,day
53100,A16,rechts,2020-01-10 07:21:00,2020-01-10 13:50:25.836719,1,January,Friday
91510,N313,beide,2022-12-12 09:44:43,2022-12-12 14:50:21.076660,1,December,Monday
50223,A15,rechts,2019-09-25 10:35:11,2019-09-25 16:20:04.022813,1,September,Wednesday
4439,N351,beide,2016-04-13 18:39:47,2016-04-13 23:51:56.760000,1,April,Wednesday
34728,N289,beide,2018-06-14 10:01:00,2018-06-14 18:30:03.738831,1,June,Thursday


In [10]:
# dataset.tijd_van_melden
dataset.to_parquet(DATA_DIR / f"prepared_y.parquet")